## Dependencies and Setup

Load the target image, and a pair of simulations to compare

In [ ]:
using Revise
using Paint
using Serialization
using Images, ImageShow
using Plots
using Evolutionary
using BlackBoxOptim
using StaticArrays
using ImageFeatures
using IntervalSets
using ReinforcementLearning
using ReinforcementLearningBase
using ReinforcementLearningZoo
using Combinatorics
using StatsBase

In [ ]:
target = float.(load("../lisa.png"))
hist1 = Serialization.deserialize("../output/simresult/simlog_100-prims_1000-batch_10-epoch_100-refine.bin")
# hist2 = Serialization.deserialize("../output/simresult/simlog_100-prims_100000-batch_10-epoch_500-refine.bin")
# hists = [hist1, hist2]
hists = [hist1]
minlen = minimum(map(hist -> length(hist.history), hists))

## Animations

Animate the states from the simulation histories, shape-by-shape.
Note the changing colour due to the re-coloring step.

In [ ]:
function drawstate(state, size)
    initial = zeros(RGB{Float32}, size) .+ state.background
    for k in eachindex(state.shapes)
        draw!(initial, state.shapes[k], RGBA{Float32}(state.colours[k].r, state.colours[k].g, state.colours[k].b, state.alpha), RasterAlgorithmScanline())
    end
    initial
end

In [ ]:
gr()
plots = []
anim_singles = @animate for i = 1:minlen+250
    ival = min(i, minlen)
    frame = []
    for hist in hists
        res = hist.history[ival]
        drawsize = size(target)
        img = drawstate(res, drawsize)
        plt = plot(img, size=drawsize, showaxis=false, ticks=false, title=string(ival-1))
        push!(frame, plt)
    end
    push!(plots, plot(frame..., layout=(1,length(frame)), size=(512,256)))
end
gif(anim_singles, "/tmp/tmp.gif", fps=12, show_msg=false)

## Loss Charts

Plot the loss of each history, as total per-step and delta between-steps.

In [ ]:
plotly()

function losses(hist)
    vals = [state.best for state in hist.history]
    diffs = [vals[i] - vals[i - 1] for i = 2:length(vals)]
    vals, diffs
end

lossset = collect(map(losses, hists))
vals = map(first, lossset)
diffs = map(last, lossset)
plot(plot(vals), plot(diffs), size=(1024, 512), legend=false)